<div align="center">
    <img src="images/um_logo.png" alt="image">
</div>

# Computación II


# ***Pipes***

# Pipes en Linux: Comunicación entre Procesos

Los pipes, conocidos también como tuberías, son una característica fundamental en los sistemas operativos tipo Unix, como Linux. Permiten la comunicación y el paso de información entre procesos. Utilizando esta técnica, la salida de un proceso puede ser utilizada directamente como entrada de otro, lo cual es una parte esencial del enfoque de programación que combina comandos pequeños pero poderosos para realizar tareas complejas.

## Conceptos Clave de los Pipes

### ¿Qué es un Pipe?

Un pipe es un canal de comunicación unidireccional que conecta la salida estándar (stdout) de un proceso con la entrada estándar (stdin) de otro. Esto permite que los programas se comuniquen entre sí sin necesidad de escribir o leer desde el sistema de archivos.

### Descriptores de Archivos y Pipes

Cada proceso en un sistema Unix-like tiene asociado un conjunto de descriptores de archivos que apuntan a archivos, dispositivos o pipes. Los descriptores de archivos 0, 1 y 2 están reservados para la entrada estándar, la salida estándar y el error estándar, respectivamente. Cuando se crea un pipe, el sistema operativo proporciona un par de descriptores de archivos: uno para leer del pipe y otro para escribir en él.

### Creación y Uso de Pipes en la Shell

Los pipes se crean en la línea de comandos utilizando el carácter `|`. Por ejemplo, el comando `ls | grep "txt"` pasa la salida del comando `ls` (que lista los archivos y directorios) a `grep`, que filtra aquellos que contienen la cadena "txt".

## Ejemplo en Línea de Comandos

Consideremos un ejemplo simple que utiliza `grep` para buscar archivos de texto que contienen una palabra específica y `wc` para contar el número de ocurrencias:

```bash
grep "palabra_clave" *.txt | wc -l


## os.pipe()
https://docs.python.org/3/library/os.html#os.pipe

# Comunicación entre Procesos con os.pipe() en Python

Python ofrece varias maneras de facilitar la comunicación entre procesos. Una de las herramientas más básicas y potentes disponibles es `os.pipe()`, proporcionada por el módulo `os`. Este método crea un par de descriptores de archivo, uno para lectura y otro para escritura, que pueden ser utilizados para la comunicación interprocesos (IPC).

## ¿Qué es os.pipe()?

`os.pipe()` es una función en Python que invoca la llamada al sistema `pipe()` disponible en sistemas operativos tipo Unix. La función `pipe()` crea un canal de comunicación unidireccional que puede ser utilizado para que los procesos se envíen datos entre sí. Este canal está implementado en el sistema operativo como un buffer de tamaño limitado.

## Cómo Funciona os.pipe()

Cuando se llama a `os.pipe()`, se devuelven dos descriptores de archivo:

- El primero (`r`) se utiliza para leer los datos que se envían a través del pipe.
- El segundo (`w`) se utiliza para escribir datos en el pipe.

Estos descriptores de archivo pueden ser utilizados directamente con otras llamadas al sistema, como `os.read()` y `os.write()`, o pueden ser encapsulados en objetos de archivo de Python utilizando `os.fdopen()` para facilitar la lectura y escritura de datos de alto nivel.

## Ejemplo de Uso de os.pipe()

El siguiente ejemplo demuestra cómo crear un pipe y utilizarlo para la comunicación entre un proceso padre y un proceso hijo creado mediante `os.fork()`.

In [ ]:
import os

# Crear un pipe
r, w = os.pipe()

# Crear un proceso hijo
pid = os.fork()
if pid > 0:
    # Proceso padre
    os.close(r)  # Cerrar el extremo de lectura en el padre
    w = os.fdopen(w, 'w')
    print("Proceso padre escribiendo")
    w.write("Hola desde el padre")
    w.close()
else:
    # Proceso hijo
    os.close(w)  # Cerrar el extremo de escritura en el hijo
    r = os.fdopen(r)
    print("Proceso hijo leyendo")
    print(r.read())
    r.close()

## Consideraciones Importantes al Usar os.pipe()

Al trabajar con `os.pipe()` en Python para la comunicación entre procesos, hay varios aspectos críticos que se deben tener en cuenta para asegurar un funcionamiento correcto y eficiente:

### Buffer de Tamaño Limitado

- El canal de comunicación creado por `os.pipe()` tiene un **buffer de tamaño limitado**. Esto significa que si el buffer se llena durante la escritura, el proceso que escribe se bloqueará hasta que haya espacio disponible. De forma similar, si el buffer está vacío, el proceso de lectura se bloqueará hasta que haya datos disponibles para leer.

### Uso de `os.close()`

- Es crucial **cerrar los descriptores de archivo adecuados** en cada proceso para evitar deadlocks y garantizar que los datos se transmitan correctamente. Cerrar los descriptores de archivo no utilizados en un proceso asegura que los recursos se liberen adecuadamente y que las señales de fin de archivo se manejen correctamente.

### Comunicación Unidireccional

- Un pipe es, por defecto, **unidireccional**. Esto significa que los datos solo pueden fluir en una dirección: del extremo de escritura al de lectura. Para lograr una comunicación bidireccional, se deben crear dos pipes, uno para cada dirección.

## ¿Por Qué Cerrar los Extremos del Pipe?

Cerrar los extremos no utilizados del pipe en los procesos padre e hijo es fundamental por varias razones:

### Prevención de Deadlocks

- Cerrar el extremo de escritura en el proceso lector (y viceversa) ayuda a **prevenir deadlocks**. Un deadlock puede ocurrir si ambos procesos esperan indefinidamente por una acción del otro, por ejemplo, si uno espera datos que nunca se escriben.

### Señalización de Fin de Archivo

- Al cerrar el extremo de escritura en el proceso que solo va a leer, se asegura que el proceso lector reciba una **señal de fin de archivo (EOF)** cuando todos los datos hayan sido leídos. Esto es crucial para que el proceso lector sepa cuándo ha terminado de recibir datos.

### Manejo Correcto de Recursos

- Cerrar los descriptores de archivo no utilizados también implica un **mejor manejo de los recursos** del sistema operativo, evitando el agotamiento de los mismos debido a descriptores de archivo abiertos innecesariamente.

### Ejemplo Práctico

Consideremos el siguiente escenario para ilustrar la importancia de cerrar los extremos adecuadamente:

- En el **proceso padre**, después de crear un pipe y bifurcar un proceso hijo, cerramos el extremo de lectura porque el padre solo escribirá datos. Esto asegura que, una vez que el padre termine de escribir y cierre el extremo de escritura, el hijo reciba una señal de EOF, indicando que no hay más datos para leer.

- En el **proceso hijo**, cerramos el extremo de escritura porque el hijo solo leerá los datos. Esto previene que el hijo (o cualquier otro proceso) escriba accidentalmente en el pipe, lo cual podría causar confusión o errores en la comunicación esperada.

Estas acciones aseguran una comunicación clara y sin ambigüedades entre los procesos, facilitando la sincronización y el intercambio de datos de manera efectiva.



# Deadlocks en Sistemas de Computación

Un **deadlock** es un estado de bloqueo en el cual dos o más procesos se quedan esperando indefinidamente por recursos o condiciones que no se pueden satisfacer, debido a una interdependencia circular entre los procesos involucrados. Este fenómeno puede ocurrir en sistemas operativos, programación concurrente, bases de datos y otros contextos donde se maneja la concurrencia y el acceso a recursos compartidos.

## Características Principales de un Deadlock

Los deadlocks son complejos y pueden ser difíciles de resolver. Se caracterizan por cuatro condiciones principales:

- **Mutua Exclusión**: Un recurso solo puede ser asignado a un proceso a la vez, lo que significa que hay recursos que no pueden ser compartidos o usados simultáneamente por varios procesos.
- **Sostener y Esperar**: Un proceso mantiene al menos un recurso y espera obtener recursos adicionales que están siendo utilizados por otros procesos.
- **No Preemptividad**: Los recursos no pueden ser forzados a ser liberados por los procesos que los tienen asignados; solo pueden ser liberados voluntariamente.
- **Espera Circular**: Existe un conjunto de procesos, cada uno de los cuales espera por un recurso que es sostenido por otro proceso en el conjunto, creando un ciclo de dependencias.

## Ejemplo de Cómo Ocurre un Deadlock

Consideremos dos procesos, `Proceso A` y `Proceso B`, y dos recursos, `Recurso 1` y `Recurso 2`. Si el `Proceso A` ocupa `Recurso 1` y espera por `Recurso 2`, mientras que el `Proceso B` ocupa `Recurso 2` y espera por `Recurso 1`, se crea una situación de espera indefinida. Ambos procesos están bloqueados, esperando por recursos que el otro proceso tiene, sin poder avanzar.

## Estrategias para la Prevención y Resolución de Deadlocks

La gestión de deadlocks implica prevenirlos, detectarlos y recuperarse de ellos:

- **Prevención**: Diseñar el sistema de tal manera que se elimine al menos una de las cuatro condiciones necesarias para que se produzca un deadlock.
- **Detección y Recuperación**: Permitir que el deadlock ocurra, detectarlo mediante algoritmos específicos y luego recuperarse. La recuperación puede implicar terminar uno o más procesos o quitarles recursos para romper el ciclo de espera.
- **Evasión**: Evaluar dinámicamente las solicitudes de recursos para asegurar que el sistema nunca entre en un estado de deadlock.

La selección de una estrategia depende de varios factores, incluidos los requisitos del sistema, las limitaciones de recursos y el impacto potencial en el rendimiento y la funcionalidad.



# Ejemplos

In [ ]:
import os

r,w = os.pipe()

r = os.fdopen(r)
w = os.fdopen(w, 'w')

msg = input('Ingrese un mensaje: ')

w.write(msg)
w.close()

print('Mensaje leido: ', r.readline())

r.close()

El script anterior utiliza la función `os.pipe()` para crear un pipe en Python, lo que facilita un canal de comunicación unidireccional entre dos puntos del programa, incluso dentro del mismo proceso. A continuación, se desglosan y explican los componentes y pasos clave del script:

1. **Creación del Pipe:**
    - Al invocar `os.pipe()`, el sistema operativo retorna un par de descriptores de archivo. Estos descriptores son identificadores de bajo nivel asignados por el sistema operativo que representan recursos, en este caso, los extremos de lectura y escritura del pipe.

2. **Conversión de Descriptores a Objetos de Archivo:**
    - Utilizando `os.fdopen()`, los descriptores de archivo se convierten en objetos de archivo de Python. Esto permite trabajar con ellos utilizando métodos de más alto nivel y más familiares para lectura y escritura. El descriptor de lectura se convierte en un objeto de archivo sin especificar el modo, lo que implica un modo de lectura predeterminado. El descriptor de escritura se abre explícitamente en modo de escritura (`'w'`).

3. **Solicitud de Entrada al Usuario:**
    - Se solicita al usuario que introduzca un mensaje mediante el uso de `input()`. El mensaje ingresado se almacenará en una variable para su posterior uso.

4. **Escritura y Cierre del Extremo de Escritura:**
    - El mensaje introducido por el usuario se escribe en el extremo de escritura del pipe. Posteriormente, es crucial cerrar este extremo con `w.close()`. Cerrar el extremo de escritura del pipe sirve como una señal para el extremo de lectura, indicando que no se enviarán más datos y que cualquier intento de lectura posterior devolverá el fin del archivo (EOF).

5. **Lectura del Mensaje:**
    - El script lee el mensaje del extremo de lectura utilizando `r.readline()`, que lee hasta encontrar una nueva línea o el fin del archivo (EOF). El fin del archivo ocurre en este contexto cuando se cierra el extremo de escritura del pipe.

6. **Cierre del Extremo de Lectura:**
    - Como buena práctica y para liberar recursos del sistema operativo adecuadamente, se cierra el extremo de lectura del pipe después de su uso con `r.close()`.

Este enfoque demuestra cómo los pipes pueden utilizarse para la comunicación entre diferentes partes de un programa en Python, aunque en este caso se aplica dentro del mismo proceso. Es especialmente útil en contextos donde se necesita comunicación entre procesos separados, facilitando un mecanismo para pasar datos de manera segura y controlada.


In [ ]:
import os


r,w = os.pipe()

pid = os.fork()

if pid:
    print('P: ',os.getpid())
    #os.close(w)
    r = os.fdopen(r)
    print('P: leyendo')
    string = r.readline()
    print('P: text = ', string)
    os.wait()

    input()
    exit()

else:
    print('H: ', os.getpid())
    #os.close(r)
    w = os.fdopen(w, 'w')
    print('H: escribiendo ')
    w.write('Texto escrito por el hijo\n')
    w.close()
    print('H: Hijo cerrado')
    input()
    exit()
    


Este script demuestra cómo establecer una comunicación entre un proceso padre y un proceso hijo en Python, utilizando `os.pipe()` para la creación de pipes y `os.fork()` para bifurcar el proceso. A continuación, se detalla el funcionamiento paso a paso del código.

## Funcionamiento del Código

1. **Importación del Módulo os**: El script inicia importando el módulo `os`, necesario para acceder a las funcionalidades del sistema operativo que permiten trabajar con pipes y procesos.

2. **Creación de un Pipe**: Se emplea `os.pipe()` para crear un canal de comunicación unidireccional. Este método devuelve un par de descriptores de archivo, `r` para lectura y `w` para escritura, que representan los extremos del pipe.

3. **Bifurcación del Proceso**: Mediante `os.fork()`, se bifurca el proceso actual en dos procesos paralelos. En el proceso padre, `os.fork()` devuelve el PID (identificador de proceso) del proceso hijo, mientras que en el proceso hijo devuelve 0.

4. **Comunicación entre Procesos**:
    - **Proceso Padre**:
        - Imprime su propio PID con `os.getpid()`.
        - Intenta leer del pipe con `r.readline()` antes de que se haya cerrado el extremo de escritura en el proceso hijo, mostrando cómo se puede esperar por datos.
        - Espera a que el proceso hijo termine su ejecución con `os.wait()`, lo cual es crucial para evitar zombies y permitir que el sistema libere los recursos asociados al hijo.
        - Espera una entrada del usuario antes de terminar, lo que permite observar la salida antes de que el script finalice.

    - **Proceso Hijo**:
        - Imprime su propio PID, diferenciándose así del proceso padre.
        - Escribe en el pipe un mensaje, utilizando `w.write()`.
        - Cierra el extremo de escritura del pipe con `w.close()`, señalizando al proceso padre que ya no se enviarán más datos.
        - Imprime un mensaje indicando que el proceso hijo se cerrará y espera una entrada del usuario antes de terminar, permitiendo ver los pasos que ejecuta antes de su conclusión.

## Notas Importantes

- El script original mantiene comentadas las líneas `os.close(r)` en el proceso hijo y `os.close(w)` en el proceso padre. Descomentar estas líneas cerraría los descriptores de archivo no utilizados en cada proceso, siguiendo buenas prácticas de programación para liberar recursos del sistema.
- El uso de `input()` antes de `exit()` en ambos procesos sirve para pausar la ejecución y facilitar la observación de la salida del script. En un escenario de producción, estas llamadas probablemente se omitirían.

Este script ofrece una visión práctica sobre cómo los pipes pueden ser utilizados para la comunicación interprocesos en Python, demostrando tanto la bifurcación de procesos con `os.fork()` como la sincronización entre procesos padre e hijo mediante pipes.
